In [1]:
from functions.functions_image_training import *
import tensorflow as tf
import timeit
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import random
%matplotlib inline

//anaconda/lib/python3.5/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [2]:
mainfolder = '/Users/carsonlam/vp/quality_confirmed/'
rejects = '/Users/carsonlam/vp/rejects/'

In [3]:
img_dict, cat_list = classifier_folder_to_jpg_dict_list(mainfolder,rejects)

In [4]:
list_of_tuples = cat_list_to_list_of_tuples(img_dict, cat_list)

In [33]:
start_time = timeit.default_timer()

############### INPUTS: including bacth size, list of tuples and image dictionary###########################
total_num = len(list_of_tuples)
classes = len(img_dict)
batch_size = 6
num_steps = int(total_num / batch_size)
single_input_shape = (450,450,3)
target_height,target_width,new_depth = single_input_shape
epochs = 1
################ TENSOR FLOW #########################################
### TF variables #####################################################
tensor_image = tf.placeholder("uint8", [None, None, 3])

############ Manipulations to tensors in TF variables ########################
#### Resizing tesors #################################################################
resized_image = tf.image.resize_images(tensor_image, target_height, target_width)

######## Manipulations to resized tensors in Resizing tensors ####################
lrflip_resized = tf.image.flip_left_right(resized_image)
udflip_resized = tf.image.flip_up_down(resized_image)

######## output count = 6
######## initialization class for session running ########################
model = tf.initialize_all_variables()
########################################################################

########## FOR TESTING ONLY ############################################
num_steps = 2


########## TRAINING LOOP #####################################################
with tf.Session() as session: 
    
    session.run(model)
    
########## NUMBER OF EPOCHS ##################################################
    for epoch in range(epochs):    
    ########## SHUFFLE TRAINING DATA BETWEEN EPOCHS #############################    
        random.shuffle(list_of_tuples)
    ########## ONE EPOCH #####################################################
        for step in range(num_steps):
            offset = (step * batch_size) % (total_num)
            start = offset
            end = offset + batch_size
            if total_num - end < batch_size:
                end += (total_num % batch_size)

    ######## INITIALIZE TRAINING BATCH ########################################################

            training_labels = np.zeros((batch_size,classes))
            training_tensors = np.zeros((batch_size,*single_input_shape))

    ####### USE 'start' and 'end' to build one BATCH #####################################
    ####### HERE ################################################################
            
            for j in range(start,end):
                
                ############ jpg to numpy at list element j ##################
                cycle_image = mpimg.imread(mainfolder + list_of_tuples[j][1])
                # j is the id for the image in this randomized dataset 
                ############ Tensor image fitting and augmentations #####################
                
                croppad_image = tf.image.resize_image_with_crop_or_pad(cycle_image, target_height, target_width)
                lrflip_croppad = tf.image.flip_left_right(croppad_image)
                udflip_croppad = tf.image.flip_up_down(croppad_image)
                
                graph_output=[resized_image,lrflip_resized,udflip_resized,croppad_image,lrflip_croppad,udflip_croppad]
                #session_output=[Resized_image,Lrflip_resized,Udflip_resized,Croppad_image,Lrflip_croppad,Udflip_croppad]
                
                session_output=session.run(graph_output,                                                        
                feed_dict={tensor_image: cycle_image})
                
                extended_batch_size = len(graph_output)
                i =0
                for output in session_output:
                    

                    training_labels[i] =  list_of_tuples[j][0]
                    training_tensors[i] = output/255 # normalize pixels 

                    i+=1
########### FEED TRAINING BATCH TO NETWORK ##########################################
########### HERE ####################################################################


            if step > 1:
                break 
#######################################################################################

    
end_time = timeit.default_timer()
print('DONE, ran for %.2fm' % ((end_time - start_time) / 60.))

[[ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]] (6, 450, 450, 3)
[[ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]] (6, 450, 450, 3)
[[ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]] (6, 450, 450, 3)
[[ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]] (6, 450, 450, 3)
[[ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]] (6, 450, 450, 3)
[[ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]] (6, 450, 450, 3)
[[ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]] (6, 450, 450, 3)
[[ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]] (6, 450, 450, 3)
[[ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]] (6, 450, 450, 3)
[[ 0.  1.  0.]
 [ 0.  1.  0.

In [30]:
type(session_output)

list